In [1]:
events = [
    ("2025-04-26T23:50:00Z", "A", 10),
    ("2025-04-29T00:30:00Z", "A", -15),
    ("2025-04-29T09:00:00Z", "B", 30),
    ("2025-04-29T10:15:00Z", "B", -10),
    ("2025-04-29T18:00:00Z", "A", 105),
    ("2025-04-29T19:00:00Z", "A", -20),
]

import pandas as pd
from datetime import datetime, timedelta
df = pd.DataFrame(events, columns=["timestamp", "sku", "delta_qty"])

In [2]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%dT%H:%M:%SZ", utc=True, errors='raise')
df['time'] = df['timestamp'].dt.time
df['day'] = df['timestamp'].dt.date

print(df)

                  timestamp sku  delta_qty      time         day
0 2025-04-26 23:50:00+00:00   A         10  23:50:00  2025-04-26
1 2025-04-29 00:30:00+00:00   A        -15  00:30:00  2025-04-29
2 2025-04-29 09:00:00+00:00   B         30  09:00:00  2025-04-29
3 2025-04-29 10:15:00+00:00   B        -10  10:15:00  2025-04-29
4 2025-04-29 18:00:00+00:00   A        105  18:00:00  2025-04-29
5 2025-04-29 19:00:00+00:00   A        -20  19:00:00  2025-04-29


In [14]:
curr_amounts = {}
final_dic = {}

df.sort_values(by='timestamp', inplace=True)

for pair, this_df in df.groupby(by=['sku','day'], sort=True):
    sku, day = pair
    try:
        current_sku = curr_amounts[sku]
    except:
        current_sku = 0
    max_sku = current_sku
    for i in range(this_df.shape[0]):
        row = this_df.iloc[i,:]
        current_sku += row['delta_qty']
        max_sku = max(max_sku, current_sku)
    curr_amounts[sku] = current_sku
    final_dic[pair] = max_sku

print(final_dic)

{('A', datetime.date(2025, 4, 26)): 10, ('A', datetime.date(2025, 4, 29)): 100, ('B', datetime.date(2025, 4, 29)): 30}


In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%dT%H:%M:%SZ", utc=True, errors='raise')
df['time'] = df['timestamp'].dt.time
df['day'] = df['timestamp'].dt.date

unique_sku = df['sku'].unique()

sku_dfs = []
for sku in unique_sku:
    sku_df = df[df['sku'] == sku].copy()
    sku_dfs.append(sku_df)

print(sku_dfs[0].head())


full_dic = {}
for i in range(len(sku_dfs)):
    sku_df = sku_dfs[i]
    sku_df.sort_values(by='timestamp' ,inplace=True)

    first_day = sku_df['day'].iloc[0]
    last_day = sku_df['day'].iloc[-1]

    curr_day = first_day
    curr_amount = 0

    sku_dic = {}

    while curr_day <= last_day:
        daily_df = sku_df[sku_df['day']==curr_day]

        max_today = curr_amount

        for j in range(daily_df.shape[0]):
            row = daily_df.iloc[j,:]
            curr_amount += row["delta_qty"]

            max_today = max(max_today, curr_amount)
        
        sku_dic[curr_day] = max_today
        curr_day = curr_day + timedelta(days=1)
    
    full_dic[unique_sku[i]] = sku_dic







                  timestamp sku  delta_qty      time         day
0 2025-04-26 23:50:00+00:00   A         10  23:50:00  2025-04-26
1 2025-04-29 00:30:00+00:00   A        -15  00:30:00  2025-04-29
4 2025-04-29 18:00:00+00:00   A        105  18:00:00  2025-04-29
5 2025-04-29 19:00:00+00:00   A        -20  19:00:00  2025-04-29
